In [1]:
from pyspark.sql import SparkSession

In [6]:
spark = SparkSession \
       .builder \
       .appName('Aula') \
       .enableHiveSupport() \
       .getOrCreate()

In [7]:
spark

In [8]:
df_aluno = spark.createDataFrame([
        {"id": 1,"nome": "fabio"},
        {"id": 2,"nome": "pedro"}
])

In [9]:
type(df_aluno)

pyspark.sql.dataframe.DataFrame

In [10]:
df_aluno.show()

+---+-----+
| id| nome|
+---+-----+
|  1|fabio|
|  2|pedro|
+---+-----+



In [11]:
##CSV HADOOP

In [14]:
df_csv = spark.read.csv('/datalake/raw/base.csv',header=True,sep=',')

In [15]:
df_csv.show()

+-------+----+---------+
|usuario|item|avaliacao|
+-------+----+---------+
|    109|   9|        3|
|    174| 412|        1|
|      7| 208|        5|
|    371|  97|        5|
|    296| 255|        2|
|    280|  82|        2|
|    271| 275|        4|
|    110| 791|        2|
|     59| 926|        1|
|    217| 576|        1|
|    145| 665|        5|
|    334| 204|        4|
|     42| 568|        4|
|    200| 143|        5|
|     89| 387|        5|
|    311| 588|        4|
|    235| 269|        4|
|    287| 156|        5|
|    344| 204|        4|
|     43| 289|        4|
+-------+----+---------+
only showing top 20 rows



In [16]:
df_json = spark.read.json('/datalake/raw/Employees100K.json')

In [17]:
df_json.count()

200000

In [18]:
df_json.printSchema()

root
 |-- Address: string (nullable = true)
 |-- Age: long (nullable = true)
 |-- DateOfJoining: string (nullable = true)
 |-- Designation: string (nullable = true)
 |-- FirstName: string (nullable = true)
 |-- Gender: string (nullable = true)
 |-- Interests: string (nullable = true)
 |-- LastName: string (nullable = true)
 |-- MaritalStatus: string (nullable = true)
 |-- Salary: string (nullable = true)
 |-- index: struct (nullable = true)
 |    |-- _index: string (nullable = true)
 |    |-- _type: string (nullable = true)



In [19]:
#S3


In [23]:
df_dados = spark.read.json('s3a://camada-bronze/dados')

In [24]:
df_dados.show()

+------------+------+-----+---------+-----------------+-------+-----+
|estado_civil|genero|idade|  produto|produto_categoria|usuario|valor|
+------------+------+-----+---------+-----------------+-------+-----+
|           0|     M|36-45|P00265242|                5|1001640| 8771|
|           0|     M|36-45|P00250342|               16|1001640| 8146|
|           0|     M|36-45|P00148542|                1|1001640| 7949|
|           0|     M|36-45|P00071242|                1|1001640|11506|
|           0|     M|26-35|P00363442|                1|1001641|15624|
|           0|     M|26-35|P00355142|                1|1001641|19000|
|           0|     M|26-35|P00350442|                2|1001641| 6630|
|           0|     M|26-35|P00024742|                8|1001641| 9823|
|           0|     M|26-35|P00226942|               11|1001641| 1715|
|           0|     M|26-35|P00184942|                1|1001641|19692|
|           0|     M|26-35|P00358242|                8|1001641| 6071|
|           0|     M

In [25]:
##BANCO DE DADOS

In [50]:
conn = 'jdbc:postgresql://postgres:5432/dvdrental'

In [51]:
prop = { 
    "user":"admin",
    "password" : "admin",
    "driver" : "org.postgresql.Driver"
}

In [37]:
df_city = spark.read.jdbc(url=conn,table='public.city',properties=prop)

In [38]:
df_city.show()

+-------+--------------------+----------+-------------------+
|city_id|                city|country_id|        last_update|
+-------+--------------------+----------+-------------------+
|      2|                Abha|        82|2006-02-15 09:45:25|
|      3|           Abu Dhabi|       101|2006-02-15 09:45:25|
|      4|                Acua|        60|2006-02-15 09:45:25|
|      5|               Adana|        97|2006-02-15 09:45:25|
|      6|         Addis Abeba|        31|2006-02-15 09:45:25|
|      7|                Aden|       107|2006-02-15 09:45:25|
|      8|               Adoni|        44|2006-02-15 09:45:25|
|      9|          Ahmadnagar|        44|2006-02-15 09:45:25|
|     10|            Akishima|        50|2006-02-15 09:45:25|
|     11|               Akron|       103|2006-02-15 09:45:25|
|     12|              al-Ayn|       101|2006-02-15 09:45:25|
|     13|           al-Hawiya|        82|2006-02-15 09:45:25|
|     14|           al-Manama|        11|2006-02-15 09:45:25|
|     15

In [39]:
df_city.printSchema()

root
 |-- city_id: integer (nullable = true)
 |-- city: string (nullable = true)
 |-- country_id: short (nullable = true)
 |-- last_update: timestamp (nullable = true)



In [52]:
query = '(select c.city_id, city , country  from public.city c \
inner join public.country c2 on c.country_id = c2.country_id ) as tab'

In [53]:
df_query = spark.read.jdbc(url=conn,table=query,properties=prop)

In [56]:
df_query.show(5)

+-------+-----------+--------------------+
|city_id|       city|             country|
+-------+-----------+--------------------+
|      2|       Abha|        Saudi Arabia|
|      3|  Abu Dhabi|United Arab Emirates|
|      4|       Acua|              Mexico|
|      5|      Adana|              Turkey|
|      6|Addis Abeba|            Ethiopia|
+-------+-----------+--------------------+
only showing top 5 rows



In [58]:
df_csv.write.json('/datalake/trusted/json/')

In [59]:
df_csv.write.parquet('/datalake/trusted/parquet/')

In [61]:
df_parquet = spark.read.parquet('/datalake/trusted/parquet')

In [62]:
df_parquet.show(5)

+-------+----+---------+
|usuario|item|avaliacao|
+-------+----+---------+
|    109|   9|        3|
|    174| 412|        1|
|      7| 208|        5|
|    371|  97|        5|
|    296| 255|        2|
+-------+----+---------+
only showing top 5 rows



In [65]:
df_csv.write.orc('/datalake/trusted/orc/', mode='append',compression='zlib')

In [66]:
df_csv.write.format('hive').saveAsTable('default.csv')

In [67]:
df_csv.write.format('hive').insertInto('default.csv2')

In [68]:
df_results = spark.read.csv('/datalake/raw/results.csv',header=True)

In [69]:
df_results.show()

+----------+----------------+-------------+--------------+--------------+--------------+---------+-------------------+-----------+
|      date|   home_teamName|away_teamName|home_scoreHome|away_scoreAway|tournamentName| cityCity|     countryCountry|neutralTRUE|
+----------+----------------+-------------+--------------+--------------+--------------+---------+-------------------+-----------+
|1872-11-30|        Scotland|      England|             0|             0|      Friendly|  Glasgow|           Scotland|      FALSE|
|1873-03-08|         England|     Scotland|             4|             2|      Friendly|   London|            England|      FALSE|
|1874-03-07|        Scotland|      England|             2|             1|      Friendly|  Glasgow|           Scotland|      FALSE|
|1875-03-06|         England|     Scotland|             2|             2|      Friendly|   London|            England|      FALSE|
|1876-03-04|        Scotland|      England|             3|             0|      Frie

In [74]:
jogos = df_results \
    .withColumnRenamed('date','dt') \
    .withColumnRenamed('home_teamName','mandante') \
    .withColumnRenamed('away_teamName','visitante') \
    .withColumnRenamed('home_scoreHome','gol_mandante') \
    .withColumnRenamed('away_scoreAway','gol_visitante')  \
    .withColumnRenamed('tournamentName','torneio') \
    .withColumnRenamed('cityCity','cidade') \
    .withColumnRenamed('countryCountry','pais') \
    .drop('neutralTRUE')

In [75]:
jogos.show(5)

+----------+--------+---------+------------+-------------+--------+-------+--------+
|        dt|mandante|visitante|gol_mandante|gol_visitante| torneio| cidade|    pais|
+----------+--------+---------+------------+-------------+--------+-------+--------+
|1872-11-30|Scotland|  England|           0|            0|Friendly|Glasgow|Scotland|
|1873-03-08| England| Scotland|           4|            2|Friendly| London| England|
|1874-03-07|Scotland|  England|           2|            1|Friendly|Glasgow|Scotland|
|1875-03-06| England| Scotland|           2|            2|Friendly| London| England|
|1876-03-04|Scotland|  England|           3|            0|Friendly|Glasgow|Scotland|
+----------+--------+---------+------------+-------------+--------+-------+--------+
only showing top 5 rows



In [76]:
jogos.printSchema()

root
 |-- dt: string (nullable = true)
 |-- mandante: string (nullable = true)
 |-- visitante: string (nullable = true)
 |-- gol_mandante: string (nullable = true)
 |-- gol_visitante: string (nullable = true)
 |-- torneio: string (nullable = true)
 |-- cidade: string (nullable = true)
 |-- pais: string (nullable = true)



In [77]:
jogos.describe().show()

+-------+----------+-------------+-------------+------------------+------------------+---------------+-------------------+-----------+
|summary|        dt|     mandante|    visitante|      gol_mandante|     gol_visitante|        torneio|             cidade|       pais|
+-------+----------+-------------+-------------+------------------+------------------+---------------+-------------------+-----------+
|  count|     40839|        40839|        40839|             40839|             40839|          40839|              40839|      40839|
|   mean|      null|         null|         null|1.7457087587844953| 1.188104507945836|           null|               null|       null|
| stddev|      null|         null|         null| 1.749144918253105|1.4051198000976632|           null|               null|       null|
|    min|1872-11-30|     Abkhazia|     Abkhazia|                 0|                 0|ABCS Tournament|6th of October City|Afghanistan|
|    max|2019-07-20|Åland Islands|Åland Islands|       

In [82]:
from pyspark.sql.functions import col
final = jogos \
    .withColumn('dt',col('dt').cast('date')) \
    .withColumn('gol_mandante',col('gol_mandante').cast('integer')) \
    .withColumn('gol_visitante',col('gol_visitante').cast('integer')) 

In [83]:
final.printSchema()

root
 |-- dt: date (nullable = true)
 |-- mandante: string (nullable = true)
 |-- visitante: string (nullable = true)
 |-- gol_mandante: integer (nullable = true)
 |-- gol_visitante: integer (nullable = true)
 |-- torneio: string (nullable = true)
 |-- cidade: string (nullable = true)
 |-- pais: string (nullable = true)



In [85]:
final.describe().show()

+-------+-------------+-------------+------------------+------------------+---------------+-------------------+-----------+
|summary|     mandante|    visitante|      gol_mandante|     gol_visitante|        torneio|             cidade|       pais|
+-------+-------------+-------------+------------------+------------------+---------------+-------------------+-----------+
|  count|        40839|        40839|             40839|             40839|          40839|              40839|      40839|
|   mean|         null|         null|1.7457087587844953| 1.188104507945836|           null|               null|       null|
| stddev|         null|         null| 1.749144918253105|1.4051198000976632|           null|               null|       null|
|    min|     Abkhazia|     Abkhazia|                 0|                 0|ABCS Tournament|6th of October City|Afghanistan|
|    max|Åland Islands|Åland Islands|                31|                21|World Unity Cup|             ‘Atele|       Éire|
+-------

In [91]:
final.persist()

DataFrame[dt: date, mandante: string, visitante: string, gol_mandante: int, gol_visitante: int, torneio: string, cidade: string, pais: string]

In [92]:
final.count()

40839

In [93]:
final.createTempView('tab')

In [96]:
spark.sql('show tables').show()

+---------+---------+-----------+
|namespace|tableName|isTemporary|
+---------+---------+-----------+
|  default|    teste|      false|
|  default|   teste2|      false|
|  default|     city|      false|
|  default|      csv|      false|
|  default|     csv2|      false|
|         |      tab|       true|
+---------+---------+-----------+



In [95]:
final.where(final.mandante == 'Brazil').show()

+----------+--------+---------+------------+-------------+------------+--------------+---------+
|        dt|mandante|visitante|gol_mandante|gol_visitante|     torneio|        cidade|     pais|
+----------+--------+---------+------------+-------------+------------+--------------+---------+
|1916-07-08|  Brazil|    Chile|           1|            1|Copa América|  Buenos Aires|Argentina|
|1916-07-12|  Brazil|  Uruguay|           1|            2|Copa América|  Buenos Aires|Argentina|
|1917-10-12|  Brazil|    Chile|           5|            0|Copa América|    Montevideo|  Uruguay|
|1919-05-11|  Brazil|    Chile|           6|            0|Copa América|Rio de Janeiro|   Brazil|
|1919-05-18|  Brazil|Argentina|           3|            1|Copa América|Rio de Janeiro|   Brazil|
|1919-05-26|  Brazil|  Uruguay|           2|            2|Copa América|Rio de Janeiro|   Brazil|
|1919-05-29|  Brazil|  Uruguay|           1|            0|Copa América|Rio de Janeiro|   Brazil|
|1919-06-01|  Brazil|Argentina

In [121]:
spark.sql('select pais from tab').show()

+-------------------+
|               pais|
+-------------------+
|           Scotland|
|            England|
|           Scotland|
|            England|
|           Scotland|
|           Scotland|
|            England|
|              Wales|
|           Scotland|
|           Scotland|
|            England|
|            England|
|              Wales|
|           Scotland|
|              Wales|
|           Scotland|
|            England|
|            England|
|              Wales|
|Republic of Ireland|
+-------------------+
only showing top 20 rows



In [119]:
final \
    .groupby('pais') \
    .agg({'pais':'count'}) \
    .orderBy('count(pais)', ascending = False) \
    .show()

+--------------------+-----------+
|                pais|count(pais)|
+--------------------+-----------+
|       United States|       1144|
|              France|        801|
|             England|        687|
|            Malaysia|        644|
|              Sweden|        637|
|             Germany|        581|
|              Brazil|        529|
|               Spain|        517|
|            Thailand|        483|
|               Italy|        480|
|         Switzerland|        477|
|             Austria|        475|
|United Arab Emirates|        472|
|        South Africa|        470|
|               Qatar|        467|
|         South Korea|        453|
|           Argentina|        449|
|             Hungary|        431|
|               Chile|        405|
|             Belgium|        396|
+--------------------+-----------+
only showing top 20 rows

